![Banner-Preparaci-n.png](https://i.postimg.cc/q7vd5LZP/Banner-Preparaci-n.png)

# Integración y preparación de datos

## L4: Consistencia

De acuerdo con el caso de uso del sector retail, vamos a cumplir los objetivos de mejorar la calidad de los datos y en este caso vamos a enfocarnos en la consistencia de los datos.

**Autor:** David Ocampo, Daniel Galindo Ruiz
    
d.ocampo@uniandes.edu.co, d.galindo@uniandes.edu.co

### 1. Contexto del negocio 

La organización está realizando el informe de ventas totales, sin embargo encuentra que la totalización de productos por categoría no concuerda con los totales esperados. Por lo tanto en necesario familiarizarse con esta dimensión de calidad de los datos y estar en capacidad de detectar problemas relacionados con esa dimensión, para poder corregirla y si es necesario notificar a la organización para que se corrija el problema de raíz.  



### Objetivos
Familiarizarse con los datos de los productos, para identificar los problemas relacionados con la consistencia y de ser posible corregirlos.
En particular se pretende:

1. Identificar registros con problemas de consistencia tanto exacta como difusa.
2. Plantear posibles causas de ese problema.
3. Plantear y aplicar alternativas de solución. 
4. Reflexionar sobre alternativas para automatizar este proceso al recibir nuevas versiones de la fuente de datos analizada.

### 2. Importación de librerías y archivos

En las siguientes líneas se importarán las librerías necesarias, las cuales son **Pandas** para el manejo de datos, la librería **distance**, particularmente la función Levenshtein y la librería **itertools**.

In [2]:
pip install python-Levenshtein

  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
  Running setup.py install for python-Levenshtein: started
  Running setup.py install for python-Levenshtein: finished with status 'error'Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [28 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\StringMatcher.py -> build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\__init__.py -> build\lib.win-amd64-3.9\Levenshtein
  running egg_info
  writing python_Levenshtein.egg-info\PKG-INFO
  writing dependency_links to python_Levenshtein.egg-info\dependency_links.txt
  deleting python_Levenshtein.egg-info\entry_points.txt
  writing namespace_packages to python_Levenshtein.egg-info\namespace_packages.txt
  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'CO


  writing requirements to python_Levenshtein.egg-info\requires.txt
  writing top-level names to python_Levenshtein.egg-info\top_level.txt
  reading manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'COPYING'
  writing manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  copying Levenshtein\_levenshtein.c -> build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\_levenshtein.h -> build\lib.win-amd64-3.9\Levenshtein
  running build_ext
  building 'Levenshtein._levenshtein' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

Encountered error while trying to install package.

python-Levenshtein

note: This is an issue with the package mentioned above, not pip.

In [4]:
# Importar las librerías necesarias según el análisis que se vaya a realizar
# Librería para manejo de datos convencional
import pandas as pd
# generar producto cruz de listas para matriz de levenshtein
import itertools
# Función de levenshtein para distancia de caracteres. Si te genera un error correr (pip install python-Leveshtein)
from Levenshtein import distance as lev

#Librería para el manejo de audio. 
import IPython.display as ipd

ModuleNotFoundError: No module named 'Levenshtein'

In [ ]:
# Te invito a escuchar el audio donde presentamos este notebook
#ipd.Audio('Consistencia.m4a') 

In [5]:
# cargar los datos en csv
data= pd.read_csv('ordenes_por_producto.csv', sep=';', encoding="utf-8")
# Visualizar los datos
data.head()

,orden_id,producto_id,vendedor_id,fecha_envio_limite,precio,valor_flete,codigo_postal_vendedor,ciudad_vendedor,departamento_vendedor,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,peso_g_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto,volumen_cm3_producto
0,1564PA53A-A,PA53,VE6785,1-ago-17,-7.27,27.12,52565,Providencia,Nariño,Electrodomésticos,19,28,29,1324,48,17,11,8976
1,28983PD80A-A,PD80,VE3342,31-may-18,3.22,6.01,52203,Colon,Nariño,Celulares,18,29,16,4677,26,6,16,2496
2,38613PB61A-A,PB61,VE7671,27-oct-17,104.18,17.70,52051,Arboleda,Nariño,Ropa_de_adultos,13,30,29,101,17,26,24,10608
3,4978PS63A-A,PS63,VE1492,13/12/2017,348.39,34.84,52323,Gualmatan,Nariño,Carnicería,33,26,13,514,47,16,43,32336
4,73781PA71A-A,PA71,VE8575,1/11/2018 0:00,29.03,30.23,5002,Abejorral,Antioquia,Electrodomésticos,27,3,13,2962,20,26,28,14560


Es necesario cumplir con la integridad de los datos, por lo tanto vamos a buscar lograr una fuente de datos con: completitud, estandarización, duplicidad y consistencia.

### 3. Análisis de la fuente de los datos

### 3.1. Registros y atributos
El número de registros y atributos nos dará una guía del correcto cargue de los datos, podemos utilizar la función shape para validar este paso.

In [6]:
# Tamaño del dataset (filas, columnas)
data.shape

(1651, 18)

In [7]:
# Revisión de los atributos
data.columns

Index(['orden_id', 'producto_id', 'vendedor_id', 'fecha_envio_limite',
       'precio', 'valor_flete', 'codigo_postal_vendedor', 'ciudad_vendedor',
       'departamento_vendedor', 'nombre_categoria_producto',
       'longitud_nombre_producto', 'longitud_descripcion_producto',
       'cantidad_fotos_producto', 'peso_g_producto', 'longitud_cm_producto',
       'altura_cm_producto', 'ancho_cm_producto', 'volumen_cm3_producto'],
      dtype='object')

### 3.2. Tipos de datos

Debemos identificar en la fuente el tipo de datos de las variables que estamos analizando, para poder establecer su formato y su dominio.

In [8]:
#identificar los tipos de datos
data.dtypes

orden_id                          object
producto_id                       object
vendedor_id                       object
fecha_envio_limite                object
precio                           float64
valor_flete                      float64
codigo_postal_vendedor             int64
ciudad_vendedor                   object
departamento_vendedor             object
nombre_categoria_producto         object
longitud_nombre_producto           int64
longitud_descripcion_producto      int64
cantidad_fotos_producto            int64
peso_g_producto                    int64
longitud_cm_producto               int64
altura_cm_producto                 int64
ancho_cm_producto                  int64
volumen_cm3_producto               int64
dtype: object

### 4. Consistencia 

Es necesario entender el dominio y estructura de los datos para esto vamos a revisar estas características para ciertos atributos que son suceptibles de un mayor entendimiento y quizás, labores de transformación.

### 4.1. Consistencia difusa en atributos de tipo string
En el perfilamiento de datos pudimos ver que las categorías de los productos tienen errores de escritura. Sin embargo, vamos a revisar con la función de Levenshtein qué tanto difieren y por qué se da ese problema. 

In [9]:
data['nombre_categoria_producto'].value_counts()

Carnicería           261
Deportes             258
Electrodomésticos    253
Frutas y verduras    249
Celulares            247
Ropa de adultos      246
Ropa_de_adultos       23
Frutas_y_verduras     12
Name: nombre_categoria_producto, dtype: int64

En un primer análisis rápido vemos que hace falta un guión bajo para la separación de palabras, pero ¿En qué medida los nombres de las categorías de productos son distintos?

In [10]:
# identificar todas las categorías
nombre_categoria=list(data['nombre_categoria_producto'].unique())
nombre_categoria

['Electrodomésticos',
 'Celulares',
 'Ropa_de_adultos',
 'Carnicería',
 'Ropa de adultos',
 nan,
 'Deportes',
 'Frutas y verduras',
 'Frutas_y_verduras']

In [15]:
# crear cruce de las categorías
producto_cruz=list(itertools.product(nombre_categoria,nombre_categoria))
producto_cruz[0:5]

[('Electrodomésticos', 'Electrodomésticos'),
 ('Electrodomésticos', 'Celulares'),
 ('Electrodomésticos', 'Ropa_de_adultos'),
 ('Electrodomésticos', 'Carnicería'),
 ('Electrodomésticos', 'Ropa de adultos')]

In [16]:
# dataframe con los cruces
matriz_leven = pd.DataFrame(producto_cruz, columns =['nombre_categoria_1', 'nombre_categoria_2'])
matriz_leven.head()

,nombre_categoria_1,nombre_categoria_2
0,Electrodomésticos,Electrodomésticos
1,Electrodomésticos,Celulares
2,Electrodomésticos,Ropa_de_adultos
3,Electrodomésticos,Carnicería
4,Electrodomésticos,Ropa de adultos


In [11]:
#aplicar función de levenshtein
for i in range(len(matriz_leven)):
    matriz_leven.at[i,'Levenshtein']=lev(str(matriz_leven.at[i,'nombre_categoria_1']),str(matriz_leven.at[i,'nombre_categoria_2']))

In [12]:
pd.crosstab(matriz_leven['nombre_categoria_1'],matriz_leven['nombre_categoria_2'],values=matriz_leven['Levenshtein'], aggfunc='sum')

nombre_categoria_2,Carnicería,Celulares,Deportes,Electrodomésticos,Frutas y verduras,Frutas_y_verduras,Ropa de adultos,Ropa_de_adultos
nombre_categoria_1,,,,,,,,
Carnicería,0.0,8.0,9.0,16.0,13.0,13.0,14.0,14.0
Celulares,8.0,0.0,6.0,15.0,14.0,14.0,12.0,12.0
Deportes,9.0,6.0,0.0,13.0,15.0,15.0,12.0,12.0
Electrodomésticos,16.0,15.0,13.0,0.0,16.0,16.0,14.0,14.0
Frutas y verduras,13.0,14.0,15.0,16.0,0.0,2.0,13.0,14.0
Frutas_y_verduras,13.0,14.0,15.0,16.0,2.0,0.0,14.0,13.0
Ropa de adultos,14.0,12.0,12.0,14.0,13.0,14.0,0.0,2.0
Ropa_de_adultos,14.0,12.0,12.0,14.0,14.0,13.0,2.0,0.0


Podemos ver que por ejemplo para la categoría Frutas_y_verduras evaluada con su par sin guión, la distancia de Levenshtein solamente es 2, por lo tanto como pensamos desde el principio, la diferencia es el caracter del guión. Esto puede ser simplemente debido a un error de digitación o de alguno de los sistemas. La solución más viable es reemplazar esos espacios vacíos por el guión para determinar las categorías reales, como lo hicimos en el tutorial de estandarización.

In [17]:
# reemplazar el guión
data['nombre_categoria_producto']=data['nombre_categoria_producto'].apply(lambda x: str(x).replace(' ','_'))
data['nombre_categoria_producto'].value_counts()

Ropa_de_adultos      269
Carnicería           261
Frutas_y_verduras    261
Deportes             258
Electrodomésticos    253
Celulares            247
nan                  102
Name: nombre_categoria_producto, dtype: int64

### 4.2. Consistencia en atributos de tipo numérico
En este escenario es un poco más fácil identificar la consistencia de los datos ya que dependiendo del dominio de los datos podemos entender cómo deberían comportarse y dónde están los posibles errores. Para este caso vamos a usar las principales estadísticas para encontrar el dominio de algunos atributos e identificar sus errores y posibles soluciones.

In [18]:
# generar estadísticas básicas de los atributos numéricos
data.describe()

,precio,valor_flete,codigo_postal_vendedor,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,peso_g_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto,volumen_cm3_producto
count,1651.000000,1549.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000,1651.000000
mean,105.351702,21.103428,48559.625681,20.017565,20.079952,19.268322,2110.227135,29.865536,18.109631,23.508783,11705.218050
std,132.496983,20.867412,12845.017466,11.741418,11.681569,11.592948,2099.076835,15.004245,11.317007,11.026377,12794.451104
min,-594.400000,0.000000,5001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.730000,6.090000,52207.000000,10.000000,10.000000,9.000000,600.500000,18.000000,9.000000,16.000000,2303.000000
50%,71.990000,15.150000,52381.000000,20.000000,20.000000,19.000000,1445.000000,30.000000,17.000000,23.000000,7695.000000
75%,151.915000,28.660000,52435.000000,30.000000,30.000000,29.000000,2932.000000,41.000000,26.000000,31.000000,16290.000000
max,771.490000,178.160000,52565.000000,40.000000,40.000000,40.000000,17034.000000,76.000000,59.000000,62.000000,90828.000000


En este caso identificamos que existen precios negativos, es una pregunta que deberíamos hacernos muy sencilla si los precios podrían ser negativos. Esta inconsistencia nos lleva a una revisión más profunda de este atributo.

In [19]:
# filtramos los precios negativos
data[data['precio']<0]['precio']

0        -7.27
8       -14.97
15     -229.74
21      -52.32
30      -55.45
         ...  
1595    -61.10
1597   -149.31
1608   -214.36
1611    -25.70
1633   -129.18
Name: precio, Length: 104, dtype: float64

Seguramente es un error de digitación, vemos que esto sucede con 104 registros. Revisando el caso con las personas de negocio, y viendo los montos involucrados representados por el total de ventas con los precios como están y luego de la transformación sugerida, llegamos a la conclusión de que es un error. Además que por alguna razón, esos registros quedaron con signo negativo, pero con el valor correcto. Es así como decidimos realizar una transformación para eliminar el signo negativo. 

In [15]:
# suma total de los productos vendidos 
data['precio'].sum()

173935.66

In [16]:
# generar una nueva columna con la transformación
data['precio_corregido']=data['precio'].apply(lambda x: abs(x))
data['precio_corregido']

0         7.27
1         3.22
2       104.18
3       348.39
4        29.03
         ...  
1646     66.10
1647     33.61
1648     40.51
1649    315.17
1650     50.26
Name: precio_corregido, Length: 1651, dtype: float64

In [17]:
# diferencia entre la suma del precio corregido y el original
data['precio_corregido'].sum()- data['precio'].sum()

20625.26000000001

In [18]:
# suma de los precios por categoría
data.groupby('nombre_categoria_producto').sum()[['precio_corregido']]

,precio_corregido
nombre_categoria_producto,
Carnicería,33614.93
Celulares,29236.88
Deportes,22909.62
Electrodomésticos,29406.08
Frutas y verduras,29431.51
Frutas_y_verduras,2920.09
Ropa de adultos,29491.17
Ropa_de_adultos,3340.96


Vemos que la diferencia entre los precios y precios corregidos es de cerca de 20.000: es un error supremamente grande por la falta de evaluación del dominio de este atributo. En el último resultado obtenido podemos revisar el informe real que necesita la compañía para tomar decisiones cruciales en temas de mercadeo y distribución.

### Caso de extensión
Trabajaremos con el archivo de productos_por_fecha, en el cual se encuentran los productos que se han vendido a lo largo del tiempo. Además de realizar los siguientes puntos para el archivo dado, contempla cuales son los riesgos que corre la empresa al utilizar datos con los problemas identificados.

+ Revisa la consistencia de la categoría de nombre_categoria_producto.
+ Revisa la consistencia del volumen de los productos.
+ Plantea alternativas de solución y en la medida de lo posible aplícalas.